Query 1 to 10

In [1]:
import sqlparse
from sqlparse.sql import Identifier, Comparison, Parenthesis, Where, IdentifierList

In [2]:
def get_collection_name(statement):
    tokens = statement.split()
    if "FROM" in tokens:
        return tokens[tokens.index("FROM") + 1]
    return None

In [3]:
def handle_select_clause(select_clause):
    fields = select_clause.split(",")
    projection = {field.strip(): 1 for field in fields}
    return {"$project": projection}

In [4]:
def handle_where_clause(where_clause):
    conditions = []
    for token in where_clause.tokens:
        if isinstance(token, Comparison):
            field = token.left.value
            operator = token.token_next(1)[1].value
            value = token.right.value
            if operator == "=":
                conditions.append({field: value})
            else:
                conditions.append({field: {f"${operator}": value}})
    return {"$match": {"$and": conditions}} if conditions else {}

In [5]:
def handle_join_clause(join_clause):
    parts = join_clause.split()
    if len(parts) >= 8 and parts[0].upper() == "JOIN":
        return {
            "$lookup": {
                "from": parts[1],
                "localField": parts[3],
                "foreignField": parts[5],
                "as": parts[7],
            }
        }
    return None

In [6]:
def handle_group_by_clause(group_by_clause):
    fields = group_by_clause.split(",")
    group = {"_id": {field.strip(): f"${field.strip()}" for field in fields}}
    return {"$group": group}

In [7]:
def handle_order_by_clause(order_by_clause):
    fields = order_by_clause.split(",")
    sort = {field.strip().split()[0]: 1 if "ASC" in field else -1 for field in fields}
    return {"$sort": sort}

In [8]:
def handle_having_clause(having_clause):
    conditions = []
    for token in having_clause.tokens:
        if isinstance(token, Comparison):
            field = token.left.value
            operator = token.token_next(1)[1].value
            value = token.right.value
            conditions.append({field: {f"${operator}": value}})
    return {"$match": {"$and": conditions}} if conditions else {}

In [9]:
def translate_sql_to_mongodb(sql_query):
    parsed = sqlparse.parse(sql_query)
    if not parsed:
        return {"error": "Invalid SQL query"}
    first_statement = parsed[0]

    collection_name = get_collection_name(first_statement.value)
    if not collection_name:
        return {"error": "Could not determine collection name"}

    pipeline = []

    where_clause = next((token for token in first_statement.tokens if isinstance(token, Where)), None)
    if where_clause:
        match_stage = handle_where_clause(where_clause)
        pipeline.append(match_stage)

    join_clause = next((token for token in first_statement.tokens if isinstance(token, Identifier)), None)
    if join_clause:
        lookup_stage = handle_join_clause(join_clause.value)
        if lookup_stage:
            pipeline.append(lookup_stage)

    group_by_clause = next((token for token in first_statement.tokens if isinstance(token, IdentifierList)), None)
    if group_by_clause:
        group_stage = handle_group_by_clause(group_by_clause.value)
        pipeline.append(group_stage)

    having_clause = next((token for token in first_statement.tokens if isinstance(token, Where)), None)
    if having_clause:
        having_stage = handle_having_clause(having_clause)
        pipeline.append(having_stage)

    order_by_clause = next((token for token in first_statement.tokens if isinstance(token, IdentifierList)), None)
    if order_by_clause:
        sort_stage = handle_order_by_clause(order_by_clause.value)
        pipeline.append(sort_stage)

    select_clause = next((token for token in first_statement.tokens if isinstance(token, IdentifierList)), None)
    if select_clause:
        projection_stage = handle_select_clause(select_clause.value)
        pipeline.append(projection_stage)

    return pipeline


In [10]:
sql_query = """
SELECT customer.name, SUM(orders.total) as total_spent
FROM customers
JOIN orders ON customers.id = orders.customer_id
WHERE customers.age > 30
GROUP BY customer.name
HAVING total_spent > 1000
ORDER BY total_spent DESC
"""

mongodb_pipeline = translate_sql_to_mongodb(sql_query)
print(mongodb_pipeline)

[{'$match': {'$and': [{'customers.age': {'$>': '30'}}]}}, {'$group': {'_id': {'customer.name': '$customer.name', 'SUM(orders.total) as total_spent': '$SUM(orders.total) as total_spent'}}}, {'$match': {'$and': [{'customers.age': {'$>': '30'}}]}}, {'$sort': {'customer.name': -1, 'SUM(orders.total)': -1}}, {'$project': {'customer.name': 1, 'SUM(orders.total) as total_spent': 1}}]


In [11]:
tpch_queries = [
    """
    SELECT l_returnflag, l_linestatus, SUM(l_quantity) AS sum_qty, SUM(l_extendedprice) AS sum_base_price
    FROM lineitem
    WHERE l_shipdate <= DATE '1998-12-01' - INTERVAL '90' DAY
    GROUP BY l_returnflag, l_linestatus
    ORDER BY l_returnflag, l_linestatus
    """,
    """
    SELECT s_acctbal, s_name, n_name, p_partkey, p_mfgr, s_address, s_phone, s_comment
    FROM part, supplier, partsupp, nation, region
    WHERE p_partkey = ps_partkey AND s_suppkey = ps_suppkey AND p_size = 15 AND p_type LIKE '%BRASS'
    AND s_nationkey = n_nationkey AND n_regionkey = r_regionkey AND r_name = 'EUROPE'
    AND ps_supplycost = (
        SELECT MIN(ps_supplycost)
        FROM partsupp, supplier, nation, region
        WHERE p_partkey = ps_partkey AND s_suppkey = ps_suppkey AND s_nationkey = n_nationkey
        AND n_regionkey = r_regionkey AND r_name = 'EUROPE'
    )
    ORDER BY s_acctbal DESC, n_name, s_name, p_partkey
    LIMIT 100
    """,
    """
    SELECT l_orderkey, SUM(l_extendedprice * (1 - l_discount)) AS revenue, o_orderdate, o_shippriority
    FROM customer, orders, lineitem
    WHERE c_mktsegment = 'BUILDING' AND c_custkey = o_custkey AND l_orderkey = o_orderkey
    AND o_orderdate < DATE '1995-03-15' AND l_shipdate > DATE '1995-03-15'
    GROUP BY l_orderkey, o_orderdate, o_shippriority
    ORDER BY revenue DESC, o_orderdate
    LIMIT 10
    """,
    """
    SELECT o_orderpriority, COUNT(*) AS order_count
    FROM orders
    WHERE o_orderdate >= DATE '1995-01-01' AND o_orderdate < DATE '1995-01-01' + INTERVAL '3' MONTH
    AND EXISTS (
        SELECT * FROM lineitem WHERE l_orderkey = o_orderkey AND l_commitdate < l_receiptdate
    )
    GROUP BY o_orderpriority
    ORDER BY o_orderpriority
    """,
    """
    SELECT n_name, SUM(l_extendedprice * (1 - l_discount)) AS revenue
    FROM customer, orders, lineitem, supplier, nation, region
    WHERE c_custkey = o_custkey AND l_orderkey = o_orderkey AND l_suppkey = s_suppkey
    AND c_nationkey = s_nationkey AND s_nationkey = n_nationkey AND n_regionkey = r_regionkey
    AND r_name = 'ASIA' AND o_orderdate >= DATE '1994-01-01' AND o_orderdate < DATE '1995-01-01'
    GROUP BY n_name
    ORDER BY revenue DESC
    """,
    """
    SELECT SUM(l_extendedprice * l_discount) AS revenue
    FROM lineitem
    WHERE l_shipdate >= DATE '1994-01-01' AND l_shipdate < DATE '1995-01-01'
    AND l_discount BETWEEN 0.06 - 0.01 AND 0.06 + 0.01 AND l_quantity < 24
    """,
    """
    SELECT supp_nation, cust_nation, l_year, SUM(volume) AS revenue
    FROM (
        SELECT n1.n_name AS supp_nation, n2.n_name AS cust_nation, EXTRACT(YEAR FROM l_shipdate) AS l_year,
        l_extendedprice * (1 - l_discount) AS volume
        FROM supplier, lineitem, orders, customer, nation n1, nation n2
        WHERE s_suppkey = l_suppkey AND l_orderkey = o_orderkey AND o_custkey = c_custkey
        AND s_nationkey = n1.n_nationkey AND c_nationkey = n2.n_nationkey
        AND (
            (n1.n_name = 'FRANCE' AND n2.n_name = 'GERMANY') OR (n1.n_name = 'GERMANY' AND n2.n_name = 'FRANCE')
        )
        AND l_shipdate BETWEEN DATE '1995-01-01' AND DATE '1996-12-31'
    ) AS shipping
    GROUP BY supp_nation, cust_nation, l_year
    ORDER BY supp_nation, cust_nation, l_year
    """,
    """
    SELECT o_year, SUM(CASE WHEN nation = 'BRAZIL' THEN volume ELSE 0 END) / SUM(volume) AS mkt_share
    FROM (
        SELECT EXTRACT(YEAR FROM o_orderdate) AS o_year, l_extendedprice * (1 - l_discount) AS volume, n2.n_name AS nation
        FROM part, supplier, lineitem, orders, customer, nation n1, nation n2, region
        WHERE p_partkey = l_partkey AND s_suppkey = l_suppkey AND l_orderkey = o_orderkey
        AND o_custkey = c_custkey AND c_nationkey = n1.n_nationkey AND n1.n_regionkey = r_regionkey
        AND s_nationkey = n2.n_nationkey AND r_name = 'AMERICA' AND p_type = 'ECONOMY ANODIZED STEEL'
        AND o_orderdate BETWEEN DATE '1995-01-01' AND DATE '1996-12-31'
    ) AS all_nations
    GROUP BY o_year
    ORDER BY o_year
    """,
    """
    SELECT c_custkey, c_name, SUM(l_extendedprice * (1 - l_discount)) AS revenue, c_acctbal, n_name, c_address, c_phone, c_comment
    FROM customer, orders, lineitem, nation
    WHERE c_custkey = o_custkey AND l_orderkey = o_orderkey AND c_nationkey = n_nationkey
    AND o_orderdate >= DATE '1993-10-01' AND o_orderdate < DATE '1994-01-01'
    AND l_returnflag = 'R'
    GROUP BY c_custkey, c_name, c_acctbal, c_phone, n_name, c_address, c_comment
    ORDER BY revenue DESC
    LIMIT 20
    """,
    """
    SELECT ps_partkey, SUM(ps_supplycost * ps_availqty) AS value
    FROM partsupp, supplier, nation
    WHERE ps_suppkey = s_suppkey AND s_nationkey = n_nationkey AND n_name = 'GERMANY'
    GROUP BY ps_partkey
    HAVING SUM(ps_supplycost * ps_availqty) > (
        SELECT SUM(ps_supplycost * ps_availqty) * 0.0001
        FROM partsupp, supplier, nation
        WHERE ps_suppkey = s_suppkey AND s_nationkey = n_nationkey AND n_name = 'GERMANY'
    )
    ORDER BY value DESC
    """
]

In [12]:

import time
for i, sql_query in enumerate(tpch_queries):

    print(f"TPC-H Query {i+1}:")
    start = time.time()
    mongodb_query = translate_sql_to_mongodb(sql_query)
    end = time.time()
    print(f"Time taken: ", (end - start)*1000," milliseconds")
    print(mongodb_query)
    print("\n" + "="*80 + "\n")

TPC-H Query 1:
Time taken:  6.883382797241211  milliseconds
[{'$match': {'$and': [{'l_shipdate': {'$<=': "DATE '1998-12-01' - INTERVAL '90' DAY"}}]}}, {'$group': {'_id': {'l_returnflag': '$l_returnflag', 'l_linestatus': '$l_linestatus', 'SUM(l_quantity) AS sum_qty': '$SUM(l_quantity) AS sum_qty', 'SUM(l_extendedprice) AS sum_base_price': '$SUM(l_extendedprice) AS sum_base_price'}}}, {'$match': {'$and': [{'l_shipdate': {'$<=': "DATE '1998-12-01' - INTERVAL '90' DAY"}}]}}, {'$sort': {'l_returnflag': -1, 'l_linestatus': -1, 'SUM(l_quantity)': -1, 'SUM(l_extendedprice)': -1}}, {'$project': {'l_returnflag': 1, 'l_linestatus': 1, 'SUM(l_quantity) AS sum_qty': 1, 'SUM(l_extendedprice) AS sum_base_price': 1}}]


TPC-H Query 2:
Time taken:  9.53984260559082  milliseconds
[{'$match': {'$and': [{'p_partkey': 'ps_partkey'}, {'s_suppkey': 'ps_suppkey'}, {'p_size': '15'}, {'p_type': {'$LIKE': "'%BRASS'"}}, {'s_nationkey': 'n_nationkey'}, {'n_regionkey': 'r_regionkey'}, {'r_name': "'EUROPE'"}, {'ps_s

In [13]:
tpch_queries = [
    """
    SELECT ps_partkey, SUM(ps_supplycost * ps_availqty) AS value
    FROM partsupp, supplier, nation
    WHERE ps_suppkey = s_suppkey AND s_nationkey = n_nationkey AND n_name = 'GERMANY'
    GROUP BY ps_partkey
    HAVING SUM(ps_supplycost * ps_availqty) > (
        SELECT SUM(ps_supplycost * ps_availqty) * 0.0001
        FROM partsupp, supplier, nation
        WHERE ps_suppkey = s_suppkey AND s_nationkey = n_nationkey AND n_name = 'GERMANY'
    )
    ORDER BY value DESC
    """,
    """
    SELECT l_shipmode, SUM(CASE WHEN o_orderpriority = '1-URGENT' OR o_orderpriority = '2-HIGH' THEN 1 ELSE 0 END) AS high_line_count,
    SUM(CASE WHEN o_orderpriority <> '1-URGENT' AND o_orderpriority <> '2-HIGH' THEN 1 ELSE 0 END) AS low_line_count
    FROM orders, lineitem
    WHERE o_orderkey = l_orderkey AND l_shipmode IN ('MAIL', 'SHIP')
    AND l_commitdate < l_receiptdate AND l_shipdate < l_commitdate AND l_receiptdate >= DATE '1994-01-01'
    AND l_receiptdate < DATE '1995-01-01'
    GROUP BY l_shipmode
    ORDER BY l_shipmode
    """,
    """
    SELECT c_count, COUNT(*) AS custdist
    FROM (
        SELECT c_custkey, COUNT(o_orderkey) AS c_count
        FROM customer LEFT OUTER JOIN orders ON c_custkey = o_custkey AND o_comment NOT LIKE '%special%requests%'
        GROUP BY c_custkey
    ) AS c_orders
    GROUP BY c_count
    ORDER BY custdist DESC, c_count DESC
    """,
    """
    SELECT 100.00 * SUM(CASE WHEN p_type LIKE 'PROMO%' THEN l_extendedprice * (1 - l_discount) ELSE 0 END) / SUM(l_extendedprice * (1 - l_discount)) AS promo_revenue
    FROM lineitem, part
    WHERE l_partkey = p_partkey AND l_shipdate >= DATE '1995-09-01' AND l_shipdate < DATE '1995-10-01'
    """,
    """SELECT s_suppkey,
s_name,
s_address,
s_phone,
total_revenue
FROM tpc.supplier
INNER JOIN (
  SELECT l_suppkey as supplier_no,  sum(l_extendedprice * (1 - l_discount)) AS total_revenue
  FROM tpc.lineitem
  WHERE l_shipdate >= date '1996-01-01'
  AND l_shipdate < date '1996-01-01' + INTERVAL '3' month
  GROUP BY l_suppkey
) AS revenue ON s_suppkey = supplier_no
WHERE total_revenue IN (
  SELECT  MAX(total_revenue)
  FROM (
    SELECT l_suppkey as supplier_no,
    SUM(l_extendedprice * (1 - l_discount)) AS total_revenue
    FROM tpc.lineitem
    WHERE l_shipdate >= date '1996-01-01'
    AND l_shipdate < date '1996-01-01' + INTERVAL '3' month
    GROUP BY l_suppkey
  ) AS revenue
)
order by s_suppkey"""
    ,
    """
    SELECT c_name, c_custkey, o_orderkey, o_orderdate, o_totalprice, SUM(l_quantity)
    FROM customer, orders, lineitem
    WHERE c_custkey = o_custkey AND o_orderkey = l_orderkey AND o_orderdate >= DATE '1995-01-01'
    AND o_orderdate < DATE '1995-04-01' AND l_returnflag = 'R'
    GROUP BY c_name, c_custkey, o_orderkey, o_orderdate, o_totalprice
    ORDER BY o_totalprice DESC, o_orderdate
    LIMIT 100
    """,
    """
    SELECT SUM(l_extendedprice * (1 - l_discount)) AS revenue
    FROM lineitem, part
    WHERE (
        p_partkey = l_partkey AND p_brand = 'Brand#12'
        AND p_container IN ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
        AND l_quantity >= 1 AND l_quantity <= 1 + 10
        AND p_size BETWEEN 1 AND 5
        AND l_shipmode IN ('AIR', 'AIR REG')
        AND l_shipinstruct = 'DELIVER IN PERSON'
    ) OR (
        p_partkey = l_partkey AND p_brand = 'Brand#23'
        AND p_container IN ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
        AND l_quantity >= 10 AND l_quantity <= 10 + 10
        AND p_size BETWEEN 1 AND 10
        AND l_shipmode IN ('AIR', 'AIR REG')
        AND l_shipinstruct = 'DELIVER IN PERSON'
    ) OR (
        p_partkey = l_partkey AND p_brand = 'Brand#34'
        AND p_container IN ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
        AND l_quantity >= 20 AND l_quantity <= 20 + 10
        AND p_size BETWEEN 1 AND 15
        AND l_shipmode IN ('AIR', 'AIR REG')
        AND l_shipinstruct = 'DELIVER IN PERSON'
    )
    """,
    """
    SELECT supp_nation, cust_nation, l_year, SUM(volume) AS revenue
    FROM (
        SELECT n1.n_name AS supp_nation, n2.n_name AS cust_nation, EXTRACT(YEAR FROM l_shipdate) AS l_year,
        l_extendedprice * (1 - l_discount) AS volume
        FROM supplier, lineitem, orders, customer, nation n1, nation n2
        WHERE s_suppkey = l_suppkey AND l_orderkey = o_orderkey AND o_custkey = c_custkey
        AND s_nationkey = n1.n_nationkey AND c_nationkey = n2.n_nationkey
        AND (
            (n1.n_name = 'FRANCE' AND n2.n_name = 'GERMANY') OR (n1.n_name = 'GERMANY' AND n2.n_name = 'FRANCE')
        )
        AND l_shipdate BETWEEN DATE '1995-01-01' AND DATE '1996-12-31'
    ) AS shipping
    GROUP BY supp_nation, cust_nation, l_year
    ORDER BY supp_nation, cust_nation, l_year
    """,
    """
    SELECT o_year, SUM(CASE WHEN nation = 'BRAZIL' THEN volume ELSE 0 END) / SUM(volume) AS mkt_share
    FROM (
        SELECT EXTRACT(YEAR FROM o_orderdate) AS o_year, l_extendedprice * (1 - l_discount) AS volume, n2.n_name AS nation
        FROM part, supplier, lineitem, orders, customer, nation n1, nation n2, region
        WHERE p_partkey = l_partkey AND s_suppkey = l_suppkey AND l_orderkey = o_orderkey
        AND o_custkey = c_custkey AND c_nationkey = n1.n_nationkey AND n1.n_regionkey = r_regionkey
        AND s_nationkey = n2.n_nationkey AND r_name = 'AMERICA' AND p_type = 'ECONOMY ANODIZED STEEL'
        AND o_orderdate BETWEEN DATE '1995-01-01' AND DATE '1996-12-31'
    ) AS all_nations
    GROUP BY o_year
    ORDER BY o_year
    """,
    """
    SELECT c_custkey, c_name, SUM(l_extendedprice * (1 - l_discount)) AS revenue, c_acctbal, n_name, c_address, c_phone, c_comment
    FROM customer, orders, lineitem, nation
    WHERE c_custkey = o_custkey AND l_orderkey = o_orderkey AND c_nationkey = n_nationkey
    AND o_orderdate >= DATE '1993-10-01' AND o_orderdate < DATE '1994-01-01'
    AND l_returnflag = 'R'
    GROUP BY c_custkey, c_name, c_acctbal, c_phone, n_name, c_address, c_comment
    ORDER BY revenue DESC
    LIMIT 20
    """,
    """
    SELECT ps_partkey, SUM(ps_supplycost * ps_availqty) AS value
    FROM partsupp, supplier, nation
    WHERE ps_suppkey = s_suppkey AND s_nationkey = n_nationkey AND n_name = 'GERMANY'
    GROUP BY ps_partkey
    HAVING SUM(ps_supplycost * ps_availqty) > (
        SELECT SUM(ps_supplycost * ps_availqty) * 0.0001
        FROM partsupp, supplier, nation
        WHERE ps_suppkey = s_suppkey AND s_nationkey = n_nationkey AND n_name = 'GERMANY'
    )
    ORDER BY value DESC
    """
]



In [14]:
import time
for i, sql_query in enumerate(tpch_queries):
    print(f"TPC-H Query {i+11}:")
    start = time.time()
    mongodb_query = translate_sql_to_mongodb(sql_query)
    end = time.time()
    print(f"Time taken: ", (end - start)*1000," milliseconds")
    print(mongodb_query)
    print("\n" + "="*80 + "\n")


TPC-H Query 11:
Time taken:  6.977558135986328  milliseconds
[{'$match': {'$and': [{'ps_suppkey': 's_suppkey'}, {'s_nationkey': 'n_nationkey'}, {'n_name': "'GERMANY'"}]}}, {'$group': {'_id': {'ps_partkey': '$ps_partkey', 'SUM(ps_supplycost * ps_availqty) AS value': '$SUM(ps_supplycost * ps_availqty) AS value'}}}, {'$match': {'$and': [{'ps_suppkey': {'$=': 's_suppkey'}}, {'s_nationkey': {'$=': 'n_nationkey'}}, {'n_name': {'$=': "'GERMANY'"}}]}}, {'$sort': {'ps_partkey': -1, 'SUM(ps_supplycost': -1}}, {'$project': {'ps_partkey': 1, 'SUM(ps_supplycost * ps_availqty) AS value': 1}}]


TPC-H Query 12:
Time taken:  7.944345474243164  milliseconds
[{'$match': {'$and': [{'o_orderkey': 'l_orderkey'}, {'l_commitdate': {'$<': 'l_receiptdate'}}, {'l_shipdate': {'$<': 'l_commitdate'}}, {'l_receiptdate': {'$>=': "DATE '1994-01-01'"}}, {'l_receiptdate': {'$<': "DATE '1995-01-01'"}}]}}, {'$group': {'_id': {'l_shipmode': '$l_shipmode', "SUM(CASE WHEN o_orderpriority = '1-URGENT' OR o_orderpriority = '2